In [6]:
import get_news
import pandas as pd

In [7]:
new_path = './news_20191220.pkl'
#df = get_news.incrental_load('./news_20191220.pkl')

# Incrementally loading data
df = pd.read_pickle(new_path)

In [10]:
df.head()

,timestamp,headline,link,content
0,"Last updated December 20, 2019 19:38:56 AEDT",‘ABSOLUTELY PETRIFIED’: Drug dealer sobs as he...,https://www.news.com.au/national/courts-law/te...,A terrified drug dealer has sobbed and called ...
1,"Last updated December 20, 2019 19:38:57 AEDT",'Do you even care?' Firey blasts ScoMo,https://www.news.com.au/technology/environment...,A fire station officer has posted an emotional...
2,"Last updated December 20, 2019 19:38:58 AEDT",Roads melting in extreme heat,https://www.news.com.au/technology/environment...,Roads in parts of South Australia are “bleedin...
3,"Last updated December 20, 2019 19:38:58 AEDT",‘Grossly immoral’: Christians lash Trump,https://www.news.com.au/finance/work/leaders/u...,A major Christian magazine has turned against ...
4,"Last updated December 20, 2019 19:38:58 AEDT",‘This is ridiculous’: Teen’s freakish act,https://www.news.com.au/sport/cricket/big-bash...,The Hobart Hurricanes looked like they were st...


In [ ]:

#%%
#df = read_news(_base_url)
import nltk
import re
from nltk.tokenize import punkt
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

sample = df.sample(1).content.iloc[0]
keep = re.compile('[a-zA-Z]')

ps = PorterStemmer()

def stem_tokenise(corpus):
    stop_words = set(stopwords.words('english'))
    return [ps.stem(w.lower()) for w in word_tokenize(corpus) if w.lower() not in stop_words and re.match(keep, w) ]


#%% TF / IDF

# TF is the number of terms for this word in this document
# IDF is the log of the number of document D over number of document contains the term
# TFIDF = TF * IDF

def tf_idf(document, documents):


#%%

words = stem_tokenise(df.iloc[0].content)
c = Counter(words)
c.most_common(10)

from wordcloud import WordCloud

wc = WordCloud(stopwords=stop_words, background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)
# %%
